In [39]:
import pandas as pd
import numpy as np

data = pd.read_csv("2019_standings_8_29.csv")

In [40]:
#Games remaining +1 - (Losses by second place team - losses by first place team)

C:\Users\jgerhartz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\jgerhartz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [42]:
def clean_data(data):
    #create list of cols for magic df
    magic_cols = ['DR','Tm','Lg','D','W','L']

    #create df with condensed cols
    magic = data[magic_cols]

    #add Div column to magic
    magic['Div'] = magic['Lg'] + magic['D']

    #add col games remaining as GR
    magic['GR'] = 162 - (magic['W'] + magic['L'])
    
    return magic

In [43]:
#might not need this
def league_rank(data, league):
    #create bool for league
    bool = data["Lg"] == league

    #create df of NL
    league_data = data[bool]
    
    #sort league_data values by wins and reset index
    league_sorted = league_data.sort_values("W", ascending = False).reset_index()
    
    #create new column league + "_Rank"
    league_sorted[league + ("_Rank")] = league_sorted.index + 1
    
    #drop index col
    #league_clean = league_sorted.drop("index")
    
    return league_sorted

In [44]:
def wildcard(data,league):   
    #create bool for non first place teams
    bool = data["DR"] != 1
    
    #create bool for league
    league_bool = data['Lg'] == league
    
    #create combo bool
    combo = bool & league_bool
    
    #create df of non first place teams
    wild = data[combo]
    
    #sort potential wildcard teams by wins and reset index
    wild_sorted = wild.sort_values("W", ascending = False).reset_index()
    
    #create new column league + "_wildcard_rank"
    wild_sorted[league + "_wildcard_rank"] = (wild_sorted.index + 1)
    
    #drop extra columns  
    wild_final = wild_sorted.drop(['DR','Lg', 'D', 'W','L','Div', 'GR','index'], axis=1)
    
    return wild_final

In [45]:
def div_magic(data,div):
    #create div bool
    div_bool = data['Div'] == div
    
    #create first bool
    first_bool = data['DR'] == 1
    
    #create second bool
    second_bool = data['DR'] == 2
     
    #create bool for first and second place in division
    first_second_bool = first_bool | second_bool
    
    #create combo bool
    combo = div_bool & first_second_bool
    
    #create combo df
    div_data = data[combo]
    
    #create MN col 
    div_data['MN'] = (div_data['GR'] + 1) - (div_data.iloc[1,5] - div_data.iloc[0,5])
    div_data.reset_index()
    
    #drop 2nd place team
    div_data.drop([div_data.index[1]], inplace = True)
    
    #remove unessasary cols
    cols = ['Tm','MN']
    mn = div_data[cols]
    

    return mn
        

In [46]:
def wildcard_magic (data, Lg):
    #create bool for Lg
    Lg_bool = data['Lg'] == Lg
    
    #create df for lg
    lg_df = data[Lg_bool]
    
    #create bool for #1 wildcard rank
    first_wc_bool = data[Lg+"_wildcard_rank"] == 1
    
    #create bool for #2 wildcard rank
    second_wc_bool = data[Lg+"_wildcard_rank"] == 2
    
    #create combo bool
    combo = first_wc_bool | second_wc_bool
    
    #create df for wildcard mns
    lg_wc_mns = lg_df[combo]
    
    #create MN col 
    lg_wc_mns['MN'] = (lg_wc_mns['GR'] + 1) - (lg_wc_mns.iloc[1,5] - lg_wc_mns.iloc[0,5])
    lg_wc_mns.reset_index()
    
    #drop 2nd place team
    sorted1 = lg_wc_mns.sort_values(Lg+"_wildcard_rank",axis=0,ascending = True)
    sorted1.drop([lg_wc_mns.index[1]])
    
    #remove unessasary cols
    cols = ['Tm','MN']
    mn = sorted1[cols]
    
    return mn

## ----Run.py Code Below----

In [47]:
al_wild_card = wildcard(magic,'AL')
nl_wild_card = wildcard(magic, 'NL')

In [48]:
merge1 = magic.merge(al_wild_card, how='left', on='Tm')
merge2 = merge1.merge(nl_wild_card, how='left', on='Tm')
#merge2.head(20)

In [49]:
#create div leader mn calculation instances
ale_mn = div_magic(merge2, "ALE")
alw_mn = div_magic(merge2, "ALW")
alc_mn = div_magic(merge2, "ALC")
nle_mn = div_magic(merge2, "NLE")
nlw_mn = div_magic(merge2, "NLW")
nlc_mn = div_magic(merge2, "NLC")

#create wild card leaders mn calculation instances
al_wc_mn = wildcard_magic(merge2, "AL")
nl_wc_mn = wildcard_magic(merge2, "NL")

C:\Users\jgerhartz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jgerhartz\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\jgerhartz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\jgerhartz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key w

In [54]:
#create list of divs with mns minus ale
mns = [alw_mn, alc_mn, nle_mn, nlw_mn, nlc_mn, al_wc_mn, nl_wc_mn]

#append div mns to one df
mns_df = ale_mn.append(mns)

In [55]:
merge3 = merge2.merge(mns_df, how='left', on='Tm')

In [56]:
merge3

,DR,Tm,Lg,D,W,L,Div,GR,AL_wildcard_rank,NL_wildcard_rank,MN
0,1,Atlanta Braves,NL,E,81,54,NLE,27,NaN,NaN,24.0
1,2,Washington Nationals,NL,E,74,58,NLE,30,NaN,1.0,28.0
2,3,Philadelphia Phillies,NL,E,69,63,NLE,30,NaN,3.0,NaN
3,4,New York Mets,NL,E,67,65,NLE,30,NaN,5.0,NaN
4,5,Miami Marlins,NL,E,47,85,NLE,30,NaN,12.0,NaN
5,1,St. Louis Cardinals,NL,C,73,59,NLC,30,NaN,NaN,29.0
6,2,Chicago Cubs,NL,C,71,61,NLC,30,NaN,2.0,28.0
7,3,Milwaukee Brewers,NL,C,68,65,NLC,29,NaN,4.0,NaN
8,4,Cincinnati Reds,NL,C,63,69,NLC,30,NaN,8.0,NaN
9,5,Pittsburgh Pirates,NL,C,56,77,NLC,29,NaN,11.0,NaN


# need to get all the divisions MN merged to one df

In [58]:
al_bool = merge3['Lg'] == "AL"
al = merge3[al_bool]
al

,DR,Tm,Lg,D,W,L,Div,GR,AL_wildcard_rank,NL_wildcard_rank,MN
15,1,New York Yankees,AL,E,88,47,ALE,27,NaN,NaN,17.0
16,2,Tampa Bay Rays,AL,E,76,58,ALE,28,2.0,NaN,32.0
17,3,Boston Red Sox,AL,E,72,62,ALE,28,4.0,NaN,NaN
18,4,Toronto Blue Jays,AL,E,54,81,ALE,27,9.0,NaN,NaN
19,5,Baltimore Orioles,AL,E,44,89,ALE,29,11.0,NaN,NaN
20,1,Minnesota Twins,AL,C,81,51,ALC,30,NaN,NaN,27.0
21,2,Cleveland Indians,AL,C,78,55,ALC,29,1.0,NaN,33.0
22,3,Chicago White Sox,AL,C,60,72,ALC,30,7.0,NaN,NaN
23,4,Kansas City Royals,AL,C,47,87,ALC,28,10.0,NaN,NaN
24,5,Detroit Tigers,AL,C,39,91,ALC,32,12.0,NaN,NaN
